# Spectral Mesh Flattening

by Ana + Gabrielle

In [58]:
import trimesh
import numpy as np

In [63]:
face = trimesh.load("Face.obj")

In [78]:
def boundary(mesh):
    """ A function to find the boundary edges and vertices of a mesh. """

    next_vertex = {}
    boundary_edges = []
    boundary_vertices = []

    edges = mesh.edges
    vertex_faces = mesh.vertex_faces

    for i in range(len(edges)):

        edge = edges[i]
        v1, v2 = edge
        v1_faces = [j for j in vertex_faces[v1] if j != -1]
        v1v2_faces = [j for j in v1_faces if j in vertex_faces[v2]]
        
        if len(v1v2_faces) == 1:
            
            next_vertex[v1] = v2
            boundary_edges  = boundary_edges + [edge]
            boundary_vertices = boundary_vertices + [v1, v2]
        
    boundary_vertices = np.unique(np.array(boundary_vertices))
    
    return boundary_edges, boundary_vertices, next_vertex

In [81]:
b_edges, b_verts, diction = boundary(face)

In [ ]:
# def distance(v1, v2):

#     return np.linalg.norm(v1 - v2)

# def circle_boundary(mesh, centre = [0, 0], radius = 1):

#     vertices = mesh.vertices
#     edges = mesh.edges

#     b_edges, b_verts = boundary(mesh)
#     total_distance = 0
#     angle_weights = []

#     circumference = 2 * np.pi * radius
    
#     for i in b_edges:

#         v1, v2 = edges[i]
#         length = distance(v1, v2)
#         total_distance += length
#         angle_weights += 

    